# Geocode New Addresses

In [ ]:
import keys
import pandas as pd
import requests
from pprint import pprint
import json
import numpy as np

In [ ]:
# Load merged, opportunities scraped
merged = pd.read_csv('merged.csv')
opportunities = pd.read_csv('opportunities_scraped.csv')

In [ ]:
merged.head()

In [ ]:
print(merged.shape)
print(merged[merged['address'].isnull() == False].shape)
print(merged[merged['address'].isnull() == True].shape)

In [ ]:
print(opportunities.shape)
print(opportunities[opportunities['scraped_address'] == 'not found'].shape)
print(opportunities[opportunities['scraped_address'] != 'not found'].shape)

In [ ]:
opportunities.head()

In [ ]:
# Copy over scraped addresses
op = opportunities[opportunities['scraped_address'] != 'not found']
print(op.shape)

In [ ]:
for i, row in op.iterrows():
    print(i, row['Opportunity ID'], row.scraped_address)
    print(merged.loc[merged['id'] == row['Opportunity ID']][['id', 'address']])

In [ ]:
for i, row in op.iterrows():
    merged.loc[merged['id'] == row['Opportunity ID'],
               'address'] = row.scraped_address

In [ ]:
# merged[merged.id=='0061a00000FqbhV'].address
print(merged.shape)
print(merged[merged['address'].isnull() == False].shape)
print(merged[merged['address'].isnull() == True].shape)

In [ ]:
# get remaining lats and longs
print(merged[merged['lat'].isnull() == False].shape)
print(merged[merged['lat'].isnull() == True].shape)

In [ ]:
key = keys.google
df = merged
# from notebook 02
counter = 0
# loop through all records with an address, but no lat/long
for i, row in df[(df['lat'].isnull() == True) &
                 (df['address'].isnull() == False)
                 #& (df['id']=='0061a00000GcQdL') #for test
                 ].iterrows():
    # get the lat and long from google maps api
    address = row.address.replace(' ', '+')
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address='+address+'=&key='+key
    response = requests.get(url)
    try:
        data = json.loads(response.text)
        # print(data)
        lat = data['results'][0]['geometry']['location']['lat']
        lng = data['results'][0]['geometry']['location']['lng']
        # add it to the dataframe
        df.loc[df['id'] == row.id, 'lat'] = lat
        df.loc[df['id'] == row.id, 'lng'] = lng
        # print(address,lat,lng)
    except:
        print('request failure for row', i, data, row.id, row.address)
    counter += 1
    if counter % 30 == 0:
        print(counter, address, lat, lng)

In [ ]:
print(merged[merged['lat'].isnull() == False].shape)
print(merged[merged['lat'].isnull() == True].shape)

In [ ]:
# Save to csv
merged.to_csv('merged_scraped.csv', index_label=False)

In [ ]:
# Delete records missing lat and long, save again

In [ ]:
merged_cleaned = merged[merged['lat'].isnull() == False]
merged_cleaned.to_csv('merged_scraped_cleaned.csv', index_label=False)